In [1]:
# IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import re

import nltk
import sys

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer,classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score, KFold,cross_validate
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)


C:\Users\P.Vijay Srinivasan\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# loading data
df = pd.read_csv("hcp_train.csv", encoding="latin-1")
df
test=pd.read_csv("hcp_test.csv",encoding="latin-1")
df

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,114933,Desktop,2,68.82.97.126,051e4a06-4cb4-4b97-b0b1-d1c6a2b8e2c2,Philadelphia,19120.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/consultat...,Cardiovascular|Clinical|False|Male|Oncology|Ps...,207RP1001X,1.0
113933,114934,Desktop,2,104.172.11.109,3289bba5-4a50-4b61-a5cc-68596bbdc2bf,Van Nuys,914014213.0,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Asthma|Arthritis|General|Clinical|Diagnosis|Rh...,2084P0800X,1.0
113934,114935,Desktop,7,174.21.94.113,9c36e90c-caa7-4323-8431-fb84f8293972,NaN,98105.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,208000000X,1.0
113935,114936,Mobile,2,69.253.129.131,afc27760-31f7-4f6e-aa08-90d8a96525d0,Wilmington,19808.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.dermatologyadvisor.com/home/topics...,pain|disease|autoimmune|multiple sclerosis|gra...,2084P0800X,1.0


In [3]:
df.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [4]:
# identified null calues and using mode to fill categorical colmns
null_columns = ["USERCITY", "USERZIPCODE", "USERAGENT"]

for i in null_columns:
    mode = df[i].mode().iloc[0]  # Extract the mode value from the Series
    df[i].fillna(mode, inplace=True)
    print(mode)

New York
63169.0
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36


In [5]:
# jsut checking for duplicates if any
df=df.drop_duplicates()
# checking null values for platformuid column to check if these rows are necessary or not 
df[df["USERPLATFORMUID"].isnull()]
# its seen that they are hcp's and crucial for model output and taxonomy prediction so we decided to keep it intact.

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
59158,60159,Desktop,2,152.11.230.114,NaN,Winston-Salem,27103.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,Urology|Chronic|Gastroenterology|Ophthalmology...,2084P0800X,1.0
80923,81924,Desktop,7,73.59.230.146,NaN,Shreveport,71106.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,207RG0300X,1.0
99416,100417,Desktop,2,152.11.230.114,NaN,Winston-Salem,27103.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,Urology|Chronic|Gastroenterology|Ophthalmology...,2084P0800X,1.0
102512,103513,Desktop,7,73.59.230.146,NaN,Shreveport,71106.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,207RG0300X,1.0


In [6]:
df.dropna(subset=["IS_HCP"], inplace=True)

In [7]:
# printing unique values
for i in df.columns:
    print(i,":",df[i].unique())
    print(" ")
    print(len(df[i].unique()))

ID : [  1001   1002   1003 ... 114935 114936 114937]
 
113936
DEVICETYPE : ['Desktop' 'Mobile' 'Tablet' 'Unknown']
 
4
PLATFORM_ID : [ 2  3  7  5  9  6  4 10 14 12  8 15  1 13 16]
 
15
BIDREQUESTIP : ['170.173.0.22' '65.216.253.25' '66.232.79.22' ... '68.82.97.126'
 '174.21.94.113' '108.41.233.175']
 
33664
USERPLATFORMUID : ['6974dcaa-f932-480e-9fb5-c52e20e1393a'
 'c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf'
 'a698de4b-e200-46dd-b5fb-40402175ae18' ...
 '9c36e90c-caa7-4323-8431-fb84f8293972'
 'afc27760-31f7-4f6e-aa08-90d8a96525d0'
 'd26887c5-15f9-4b72-80dc-728039529e60']
 
53781
USERCITY : ['Portland' 'Arlington' 'New Meadows' ... 'Texhoma' 'Goodhue' 'Gambier']
 
4420
USERZIPCODE : [9.72060000e+04 2.22020000e+04 8.36540000e+04 ... 4.34500000e+04
 7.87356202e+08 7.60865488e+08]
 
11278
USERAGENT : ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like

In [8]:
df=df.drop("CHANNELTYPE",axis=1)
# only one value this column does not provide any useful information for the analysis. 
# Having a single value throughout the entire column does not contribute to the variability of the data and 
# does not provide any discriminatory power.

In [9]:
# doing same opearation for test data
test=test.drop("CHANNELTYPE",axis=1)
test

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,URL,KEYWORDS
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,115504,Desktop,7,71.105.120.171,3e2578c8-f794-41af-a38c-c5cfb3c0f014,Brooklyn,11226.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...
4,115505,Desktop,2,73.82.211.73,ec2ae7ce-6a8c-4156-98a7-07203e60f483,Marietta,30062.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...
...,...,...,...,...,...,...,...,...,...,...,...
28488,143989,Desktop,2,69.202.233.241,78ce4bbe-3885-4c14-b945-a0ea1e4574f4,Brooklyn,11215.0,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,Online Medical Journal,https://www.renalandurologynews.com/home/news/...,Transplantation|Psychiatry|Heart Failure|Angio...
28489,143990,Desktop,7,75.4.190.65,a53799e1-b279-40cf-b467-11b4d659e325,Miami,33178.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,https://www.cureus.com/publish/articles/144818...,small|hospitals|nephrology|emergency medicine|...
28490,143991,Desktop,7,137.52.180.45,acc732a8-2ee9-4e2c-9933-70e34db48101,Fort Lauderdale,33314.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,https://www.cureus.com/registrations/continue?...,small|hospitals|nephrology|emergency medicine|...
28491,143992,Desktop,8,66.249.66.4,dce21294-b105-4abb-b145-4e62d71def44,NaN,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal,https://radrounds.com/radiology-case-images-te...,the|disease|pain|radiology|lung abscess|dyspar...


In [10]:
# removing unnecessary columns in train and test dataset
cols_to_remove = ["USERCITY", "USERZIPCODE", "USERAGENT", "PLATFORM_ID", "BIDREQUESTIP","USERPLATFORMUID"]
df = df.drop(cols_to_remove, axis=1)

In [11]:
test=test.drop(cols_to_remove, axis=1)

In [12]:
test

,ID,DEVICETYPE,PLATFORMTYPE,URL,KEYWORDS
0,115501,Desktop,Online Medical Journal,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,Online Medical Journal,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,Online Medical Journal,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,115504,Desktop,Online Medical Journal,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...
4,115505,Desktop,Online Medical Journal,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...
...,...,...,...,...,...
28488,143989,Desktop,Online Medical Journal,https://www.renalandurologynews.com/home/news/...,Transplantation|Psychiatry|Heart Failure|Angio...
28489,143990,Desktop,Online Medical Journal,https://www.cureus.com/publish/articles/144818...,small|hospitals|nephrology|emergency medicine|...
28490,143991,Desktop,Online Medical Journal,https://www.cureus.com/registrations/continue?...,small|hospitals|nephrology|emergency medicine|...
28491,143992,Desktop,Online Medical Journal,https://radrounds.com/radiology-case-images-te...,the|disease|pain|radiology|lung abscess|dyspar...


In [13]:
df["URL"][0]

'https://www.cancertherapyadvisor.com/home/cancer-topics/prostate-cancer/'

In [14]:
df.iloc[3][4]

'Dental|Total|Clinical|Pharmacology|Physicians|Health|Drug Therapy|Oncology|False'

In [15]:
# URL keywords extraction for both train and test datasets
from urllib.parse import urlparse

common_words = ['http:', 'https:', 'www', 'com', 'home', 'html', 'htm', 'php', 'asp', 'aspx', 'jsp', 'index', 'default', 'blog', 'article', 'articles', 'category', "#","39455",'after','a','to','the']
custom_stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def extract_keywords(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc.replace('www.', '')
    path = parsed_url.path.strip('/')

    subdomains = parsed_url.hostname.split('.')
    subdomains = [subdomain for subdomain in subdomains if subdomain.lower() not in common_words]

    path_keywords = [word.lower() for word in re.split('[/-]', path) if word and word.lower() not in common_words]
    path_keywords = [token for token in path_keywords if re.match(r'^[a-zA-Z0-9]+$', token)]
    path_keywords = [lemmatizer.lemmatize(token) for token in path_keywords]

    domain_keywords = [word.lower() for word in re.split('[/-]', domain) if word and word.lower() not in common_words]
    domain_keywords = [token for token in domain_keywords if re.match(r'^[a-zA-Z0-9]+$', token)]
    domain_keywords = [lemmatizer.lemmatize(token) for token in domain_keywords]

    
    keywords = subdomains + domain_keywords + path_keywords
    
    keywords_strings = ', '.join(keywords)
    
    return keywords_strings


In [16]:
def remove_duplicates(keywords_string, custom_stopwords):
    tokens = keywords_string.split()
    unique_tokens = set()

    for token in tokens:
        lowercase_token = token.lower()
        if lowercase_token not in custom_stopwords:
            unique_tokens.add(lowercase_token)

    return ' '.join(unique_tokens)

In [17]:
# removing duplicate or repeating keywords in URL'S
df['url_terms'] = df['URL'].apply(lambda x: remove_duplicates(extract_keywords(x),custom_stopwords))
test["url_terms"] = test['URL'].apply(lambda x: remove_duplicates(extract_keywords(x),custom_stopwords))

In [18]:
df["KEYWORDS"][113932]

'Cardiovascular|Clinical|False|Male|Oncology|Psychiatry|Medicine|Hepatology|General|Total|Gastroenterology|Neurology|Urology|Health|Cardiology|Asthma|Orthopedics|Pediatrics|Rheumatology|Dermatology|Surgery|Acquired|Ophthalmology'

In [19]:
df["url_terms"][108]
test["url_terms"][199]

'duct, cureus, biliary, hamartoma, review 98184, systematic,'

In [20]:
df["url_terms"][113932]
df["url_terms"].head(1)

0    prostate, cancertherapyadvisor, topic, cancer,...
Name: url_terms, dtype: object

In [21]:
len(set(df["url_terms"]))

2651

In [22]:
#removing symbols from KEYWORDS column
def preprocess_keywords(row):
    keywords = row['KEYWORDS']
    # Remove special characters (except '|') using regular expressions
    keywords = re.sub(r'[^\w\s|]', '', keywords)
    return keywords

# Apply preprocessing to the 'KEYWORDS' column
df['PREPROCESSED_KEYWORDS'] = df.apply(preprocess_keywords, axis=1)
test['PREPROCESSED_KEYWORDS'] = df.apply(preprocess_keywords, axis=1)

In [23]:
df['PREPROCESSED_KEYWORDS'][113932]

'Cardiovascular|Clinical|False|Male|Oncology|Psychiatry|Medicine|Hepatology|General|Total|Gastroenterology|Neurology|Urology|Health|Cardiology|Asthma|Orthopedics|Pediatrics|Rheumatology|Dermatology|Surgery|Acquired|Ophthalmology'

In [24]:
# USING WORD2VEC MODEL ON KEYWORDS COLUMN FOR NUMBERICAL VECTOR REPRESENATION
from gensim.models import Word2Vec

sentences = [keywords.split('|') for keywords in df['KEYWORDS']]
test_sentences = [keywords.split('|') for keywords in test['KEYWORDS']]
# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1, epochs=10)
model1=Word2Vec(test_sentences, vector_size=100, window=5, min_count=1, sg=1, epochs=10)

In [25]:
# saving models
model.save("my_word2vec_model.bin")
model1.save("my_word2vec_model1.bin")

In [26]:
# CREATING VECTOR REPRESENTATION FOR ALL KEYWORDS
keyword_vector = {}
keyword_vector1 = {}

def get_keyword_vector(row, model):
    keywords = row['PREPROCESSED_KEYWORDS']
    tokens = keywords.split('|')
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        for i in tokens:
            keyword_vector[i] = sum(vectors) / len(vectors)  # Assign the vector to the dictionary
            return keyword_vector[i]
        else:
            return None
def get_keyword_vector1(row, model):
    keywords = row['PREPROCESSED_KEYWORDS']
    tokens = keywords.split('|')
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        for i in tokens:
            keyword_vector1[i] = sum(vectors) / len(vectors)  # Assign the vector to the dictionary
            return keyword_vector1[i]
        else:
            return None

# Load the Word2Vec model
model = Word2Vec.load("my_word2vec_model.bin")
model1=Word2Vec.load("my_word2vec_model1.bin")
# Example usage
df['KEYWORD_VECTORS'] = df.apply(lambda row: get_keyword_vector(row, model), axis=1)
test['KEYWORD_VECTORS'] =test.apply(lambda row: get_keyword_vector1(row, model1), axis=1)

In [27]:
# keyword vector for train data
keyword_vector

{'General': array([ 0.00952216,  0.25379446, -0.16179103, -0.19294918,  0.07659448,
        -0.1430727 ,  0.2494011 ,  0.59941   , -0.18976498,  0.10594461,
         0.12584627, -0.34417748,  0.03482585,  0.18309507,  0.28656474,
         0.16583209,  0.00095265, -0.22628681, -0.01223911, -0.42210796,
         0.2196133 ,  0.09448147,  0.1449183 , -0.16190885, -0.24836211,
         0.01104635, -0.3959586 , -0.03450416,  0.13217625,  0.19418116,
         0.32883823, -0.25340608, -0.07715938, -0.0991968 , -0.35176125,
        -0.09180455, -0.0027289 , -0.19495292,  0.17904225, -0.33751765,
         0.4143009 ,  0.04605953,  0.07010729,  0.26502532,  0.04606257,
         0.07643953, -0.3757261 , -0.09475878,  0.10642264,  0.11932822,
         0.02770224, -0.02421908, -0.03715911, -0.04329406, -0.05133732,
        -0.06510667,  0.46155348, -0.02078337, -0.03297124,  0.21149626,
         0.01588492,  0.01093107, -0.11233635, -0.1345603 , -0.30457014,
         0.5173763 ,  0.09578189,  0.208

In [28]:
# keyword vector for train data
keyword_vector1

{'General': array([-1.53359026e-01,  2.63597727e-01,  7.62116611e-02,  2.59216189e-01,
        -1.09721810e-01, -3.19298804e-01,  2.06216604e-01,  7.16670692e-01,
        -4.38138723e-01, -1.81095243e-01, -1.16925687e-02, -2.19912082e-01,
        -1.17437094e-01,  1.65260255e-01,  3.06070354e-02,  1.88617244e-01,
         4.53775406e-01, -8.14606696e-02, -1.13437094e-01, -5.65823793e-01,
        -5.78370951e-02,  3.73594463e-02, -1.09895609e-01, -1.42381161e-01,
        -3.14455539e-01,  2.08956987e-01, -2.41423815e-01, -2.65868623e-02,
        -8.21533501e-02,  1.05852187e-01,  3.27171177e-01, -1.06174439e-01,
         9.20763537e-02, -1.67192861e-01, -2.59081423e-01,  1.23459615e-01,
         1.31964490e-01,  7.13930577e-02, -4.10379767e-02, -2.86897123e-01,
         1.92068815e-01, -1.17957592e-01, -1.72745749e-01,  1.76445737e-01,
         6.88833743e-02, -1.86321080e-01, -3.77394736e-01, -3.17299535e-04,
         4.75306846e-02, -1.53287351e-01, -1.80419777e-02, -3.03123165e-02,
 

In [29]:
df['KEYWORD_VECTORS']

0         [-0.1105455, 0.32152465, -0.13879313, -0.16020...
1         [-0.103736594, 0.3792737, -0.1874648, -0.18215...
2         [-0.1105455, 0.32152465, -0.13879313, -0.16020...
3         [-0.21777728, 0.22715868, -0.009386333, -0.141...
4         [0.041507114, 0.25298634, -0.087673865, -0.139...
                                ...                        
113932    [-0.02299761, 0.33383617, -0.13396375, -0.1392...
113933    [-0.031221017, 0.2999795, -0.15252496, -0.1561...
113934    [-0.004093866, 0.3178835, -0.1022364, -0.11286...
113935    [0.11825936, 0.2481463, -0.022817282, 0.078058...
113936    [-0.094351746, 0.3770146, -0.09252918, -0.1998...
Name: KEYWORD_VECTORS, Length: 113936, dtype: object

In [30]:
df["PREPROCESSED_KEYWORDS"][113932]

'Cardiovascular|Clinical|False|Male|Oncology|Psychiatry|Medicine|Hepatology|General|Total|Gastroenterology|Neurology|Urology|Health|Cardiology|Asthma|Orthopedics|Pediatrics|Rheumatology|Dermatology|Surgery|Acquired|Ophthalmology'

In [31]:
df

,ID,DEVICETYPE,PLATFORMTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP,url_terms,PREPROCESSED_KEYWORDS,KEYWORD_VECTORS
0,1001,Desktop,Online Medical Journal,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,"prostate, cancertherapyadvisor, topic, cancer,...",General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.1105455, 0.32152465, -0.13879313, -0.16020..."
1,1002,Desktop,Online Medical Journal,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0,"hematology, support, cancertherapyadvisor, med...",Bone Marrow|Radiography|Chronic|Oncology|Psych...,"[-0.103736594, 0.3792737, -0.1874648, -0.18215..."
2,1003,Desktop,Online Medical Journal,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0,"prostate, cancertherapyadvisor, topic, cancer,...",General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.1105455, 0.32152465, -0.13879313, -0.16020..."
3,1004,Desktop,Online Medical Journal,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0,"globalrph, maintenance, consult, dose, warfari...",Dental|Total|Clinical|Pharmacology|Physicians|...,"[-0.21777728, 0.22715868, -0.009386333, -0.141..."
4,1005,Mobile,Online Medical Journal,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0,"cureus, triple, cystic, fibrosis, of, conducta...",Critical Care|Emergency Medicine|General Pract...,"[0.041507114, 0.25298634, -0.087673865, -0.139..."
...,...,...,...,...,...,...,...,...,...,...
113932,114933,Desktop,Online Medical Journal,https://www.clinicaladvisor.com/home/consultat...,Cardiovascular|Clinical|False|Male|Oncology|Ps...,207RP1001X,1.0,"consultation, monogamous, risk, clinicaladviso...",Cardiovascular|Clinical|False|Male|Oncology|Ps...,"[-0.02299761, 0.33383617, -0.13396375, -0.1392..."
113933,114934,Desktop,Online Medical Journal,https://www.clinicaladvisor.com/home/features/...,Asthma|Arthritis|General|Clinical|Diagnosis|Rh...,2084P0800X,1.0,"derm, clinicaladvisor, feature, dx",Asthma|Arthritis|General|Clinical|Diagnosis|Rh...,"[-0.031221017, 0.2999795, -0.15252496, -0.1561..."
113934,114935,Desktop,Online Medical Journal,https://www.cureus.com/articles/39455-whats-th...,Rehabilitation|Transplantation|Clinical|Preven...,208000000X,1.0,"cureus, ratio hazard, and, risk, differentiati...",Rehabilitation|Transplantation|Clinical|Preven...,"[-0.004093866, 0.3178835, -0.1022364, -0.11286..."
113935,114936,Mobile,Online Medical Journal,https://www.dermatologyadvisor.com/home/topics...,pain|disease|autoimmune|multiple sclerosis|gra...,2084P0800X,1.0,"for, autoimmune, topic, dermatitis, risk, derm...",pain|disease|autoimmune|multiple sclerosis|gra...,"[0.11825936, 0.2481463, -0.022817282, 0.078058..."


In [32]:
# creating a taxonomy map to fill taxonomies where same keywords matches
keyword_taxonomy_map = {}
taxonomy_map={}
url_map={}
for index, row in df[df["IS_HCP"]==1].iterrows():
    keywords = row['KEYWORDS']
    taxonomy = row['TAXONOMY']
    url = row['URL']
    if url not in url_map:
        url_map[url] = taxonomy
    keyword_list = keywords.split('|')

    for keyword in keywords.split('|'):
        if keyword not in keyword_taxonomy_map:
            keyword_taxonomy_map[keyword] = [taxonomy]  # Create a new list with the first taxonomy
        else:
            if taxonomy not in keyword_taxonomy_map[keyword]:
                keyword_taxonomy_map[keyword].append(taxonomy)

        if keywords not in taxonomy_map:
            taxonomy_map[keywords] = taxonomy
# print(keyword_taxonomy_map)
taxonomy_map

{'Dental|Total|Clinical|Pharmacology|Physicians|Health|Drug Therapy|Oncology|False': '2084P0800X',
 'Intensive Care|Shock|Ischemic Heart Disease|Health|Drainage|Liver Transplantation|Transplantation|Controlled|Urology|Critical Care|Erythema|Malnutrition|Hepatology|Hypotension|General|Intrahepatic|Small|Neurology|Lung|Cardiac|Oral|Chronic|Total|Tachycardia|Liver|Angiotensin|Kidney Transplantation|Diagnosis|Endocrine|False|Nervous System|Cardiology|Critical Care Medicine|Ophthalmology|Resuscitation|Adrenal Insufficiency|Intravenous|Metabolic|Gastroenterology|Cardiac Output|Hemorrhage|Diabetes Mellitus|Clinical|Induced|Oncology|Psychiatry|Medicine|Hematuria|Myocardial Ischemia|Respiratory Tract|Intrinsic|Nephritis|Rheumatology|Perfusion|Cardiomyopathy|Refractory|Large|Dermatology': '2084P0804X',
 'Gastrointestinal Tract|Female|Oncology|Medicine|Small|Metabolic|Elderly|Health|False|Trauma|Total|Lung|Male|Aged|Liver|Drug Therapy|Dental': '2084N0400X',
 'Rehabilitation|Transplantation|Clinic

In [33]:
# creating keyword taxonomy map to find similar keywords using similarity techniques for the kywods that doesnt match exactly
keyword_taxonomy_map

{'Dental': ['2084P0800X',
  '2084N0400X',
  '207R00000X',
  '2084P0804X',
  '207Q00000X',
  '363AM0700X',
  '207RC0000X',
  '207N00000X',
  '390200000X',
  '174400000X',
  '207T00000X',
  '363A00000X',
  '2083X0100X',
  '2084P2900X',
  '363LF0000X',
  '207RH0000X',
  '208D00000X',
  '208000000X',
  '207W00000X',
  '364SP0809X',
  '207RG0300X',
  '2084S0012X',
  '2084F0202X',
  '207RX0202X',
  '207RH0003X',
  '207P00000X',
  '207YX0901X',
  nan,
  '207RE0101X',
  '207RR0500X',
  '204D00000X',
  '207RP1001X',
  '2084N0008X',
  '207RG0100X',
  '163W00000X',
  '207VM0101X',
  '207RA0401X',
  '2084N0402X',
  '208M00000X',
  '2084N0600X',
  '2084P0015X',
  '2080P0216X',
  '2084P0802X',
  '2085R0202X',
  '363LP0200X',
  '2084V0102X',
  '207RA0000X',
  '2080P0207X',
  '2084A0401X',
  '363L00000X',
  '207V00000X',
  '363LA2200X',
  '2084P0805X',
  '207X00000X',
  '208VP0000X',
  '363LP0808X',
  '207RC0200X',
  '207ZN0500X',
  '2080P0206X',
  '207ZP0213X',
  '2084A2900X',
  '2080P0008X',
  '2085

In [34]:
# classification model training
# dropping url and keywords column from test
test=test.drop(["URL","KEYWORDS"],axis=1)

In [35]:
keyword_vector 

{'General': array([ 0.00952216,  0.25379446, -0.16179103, -0.19294918,  0.07659448,
        -0.1430727 ,  0.2494011 ,  0.59941   , -0.18976498,  0.10594461,
         0.12584627, -0.34417748,  0.03482585,  0.18309507,  0.28656474,
         0.16583209,  0.00095265, -0.22628681, -0.01223911, -0.42210796,
         0.2196133 ,  0.09448147,  0.1449183 , -0.16190885, -0.24836211,
         0.01104635, -0.3959586 , -0.03450416,  0.13217625,  0.19418116,
         0.32883823, -0.25340608, -0.07715938, -0.0991968 , -0.35176125,
        -0.09180455, -0.0027289 , -0.19495292,  0.17904225, -0.33751765,
         0.4143009 ,  0.04605953,  0.07010729,  0.26502532,  0.04606257,
         0.07643953, -0.3757261 , -0.09475878,  0.10642264,  0.11932822,
         0.02770224, -0.02421908, -0.03715911, -0.04329406, -0.05133732,
        -0.06510667,  0.46155348, -0.02078337, -0.03297124,  0.21149626,
         0.01588492,  0.01093107, -0.11233635, -0.1345603 , -0.30457014,
         0.5173763 ,  0.09578189,  0.208

In [36]:
# Creating a feature matrix for modelling
keyword_vector_length = len(df['KEYWORD_VECTORS'].iloc[0])
vector_length = keyword_vector_length  # Assuming 'vector_length' is the correct length of the keyword vectors

# Adjust the shape of the feature matrix
feature_matrix = np.zeros((len(df), keyword_vector_length + vector_length))

# Populate the feature matrix
for i, keywords in enumerate(df['PREPROCESSED_KEYWORDS']):
    keyword_vectors = []  # Initialize the keyword vectors list for each instance
    keywords_split = keywords.split("|")
    for keyword in keywords_split:
        if keyword in keyword_vector:
            vector = keyword_vector[keyword]  # Retrieve the keyword vector from the mapping
            keyword_vectors.append(vector)  # Add the 'KEYWORD_VECTORS' value to the feature matrix
    if keyword_vectors:
        feature_matrix[i, :keyword_vector_length] = np.mean(keyword_vectors, axis=0)  # Use mean pooling for keyword vectors
    feature_matrix[i, keyword_vector_length:] = df['KEYWORD_VECTORS'].iloc[i]
feature_matrix[11]

array([-2.85085626e-02,  2.83376098e-01, -1.23169295e-01, -1.81398079e-01,
        1.87270105e-01, -1.93438053e-01,  3.57721150e-01,  5.85316241e-01,
       -2.41895422e-01, -1.41944773e-02,  1.38384134e-01, -2.59801447e-01,
        2.27863109e-03,  9.53481495e-02,  2.84678787e-01,  1.27645269e-01,
        6.60104454e-02, -1.54340297e-01, -1.04294613e-01, -4.68824536e-01,
        2.53452122e-01,  5.64458631e-02,  5.81149682e-02, -1.69409990e-01,
       -1.02148220e-01, -7.76765868e-02, -2.99392402e-01, -5.41238599e-02,
        1.45334423e-01,  2.06113994e-01,  2.70572782e-01, -3.18851352e-01,
       -9.52508152e-02, -1.65962398e-01, -2.71461368e-01, -5.23454584e-02,
        1.92899369e-02, -2.43720263e-01,  1.72289029e-01, -3.26596916e-01,
        3.56716812e-01,  2.42894012e-02,  8.48625749e-02,  2.70758510e-01,
        1.93803057e-01,  6.44581616e-02, -2.22216755e-01, -7.93306008e-02,
        1.02368757e-01,  9.33336318e-02,  9.04023573e-02, -4.07944657e-02,
        5.68098649e-02, -

In [37]:
# feature matrix for test data
keyword_vector_length1 = len(test['KEYWORD_VECTORS'].iloc[0])
vector_length1 = keyword_vector_length1  # Assuming 'vector_length1' is the correct length of the keyword vectors

# Adjust the shape of the feature matrix
feature_matrix1 = np.zeros((len(test), keyword_vector_length1 + vector_length1))

# Populate the feature matrix
for i, keywords in enumerate(test['PREPROCESSED_KEYWORDS']):
    keyword_vectors1 = []  # Initialize the keyword vectors list for each instance
    keywords_split = keywords.split("|")
    for keyword in keywords_split:
        if keyword in keyword_vector1:
            vector = keyword_vector1[keyword]  # Retrieve the keyword vector from the mapping
            keyword_vectors1.append(vector)
    
    if keyword_vectors1:
        feature_matrix1[i, :keyword_vector_length1] = np.mean(keyword_vectors1, axis=0)  # Use mean pooling for keyword vectors
    feature_matrix1[i, keyword_vector_length1:] = test['KEYWORD_VECTORS'].iloc[i]  # Add the 'KEYWORD_VECTORS' value to the feature matrix

feature_matrix1.shape 

(28493, 200)

In [38]:
# added all necessary columns into feature matrix for modelling purposes
one_hot_encoder = OneHotEncoder(sparse=False)
device_type_encoded = one_hot_encoder.fit_transform(df['DEVICETYPE'].values.reshape(-1, 1))

label_encoder = LabelEncoder()
platform_type_encoded = label_encoder.fit_transform(df['PLATFORMTYPE'])

url_terms_counts = df['url_terms'].value_counts().to_dict()
df['url_terms_encoded'] = df['url_terms'].map(url_terms_counts)

additional_features = np.concatenate([device_type_encoded, platform_type_encoded.reshape(-1, 1), df['url_terms_encoded'].values.reshape(-1, 1)], axis=1)

feature_matrix = np.concatenate([feature_matrix, additional_features], axis=1)


feature_matrix.shape

C:\Users\P.Vijay Srinivasan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(113936, 206)

In [55]:
feature_matrix

array([[-5.40496223e-02,  2.96569914e-01, -1.13387406e-01, ...,
         0.00000000e+00,  3.00000000e+00,  7.92000000e+02],
       [-6.69821873e-02,  2.86478817e-01, -1.27119541e-01, ...,
         0.00000000e+00,  3.00000000e+00,  2.24600000e+03],
       [-5.40496223e-02,  2.96569914e-01, -1.13387406e-01, ...,
         0.00000000e+00,  3.00000000e+00,  7.92000000e+02],
       ...,
       [-4.32764366e-02,  2.95503765e-01, -1.17948055e-01, ...,
         0.00000000e+00,  3.00000000e+00,  1.51300000e+03],
       [ 1.06191538e-01,  2.57609129e-01, -1.91951655e-02, ...,
         0.00000000e+00,  3.00000000e+00,  1.03000000e+02],
       [-6.73266947e-02,  2.96581984e-01, -1.19982637e-01, ...,
         0.00000000e+00,  3.00000000e+00,  1.92300000e+03]])

In [39]:
# # added all necessary columns into feature matrix for test data
one_hot_encoder1 = OneHotEncoder(sparse=False)
device_type_encoded1 = one_hot_encoder1.fit_transform(test['DEVICETYPE'].values.reshape(-1, 1))

# Label Encoding for 'platform_type'
label_encoder1 = LabelEncoder()
platform_type_encoded1 = label_encoder1.fit_transform(test['PLATFORMTYPE'])

# Count Encoding for 'url_terms'
url_terms_counts1 = test['url_terms'].value_counts().to_dict()
test['url_terms_encoded'] = test['url_terms'].map(url_terms_counts1)

# Combine encoded features and create the feature matrix
additional_features1= np.concatenate([device_type_encoded1, platform_type_encoded1.reshape(-1, 1), test['url_terms_encoded'].values.reshape(-1, 1)], axis=1)

# Update the feature matrix by concatenating with additional features
feature_matrix1 = np.concatenate([feature_matrix1, additional_features1], axis=1)
# feature_matrix1 =feature_matrix1.concatenate([device_type_encoded1, platform_type_encoded1.reshape(-1, 1), test['url_terms_encoded'].values.reshape(-1, 1)], axis=1)

# Split the test into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(feature_matrix, test['is_hcp'], test_size=0.2, random_state=42)

# Train your classification model using the prepared test
# ...

feature_matrix1

C:\Users\P.Vijay Srinivasan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[-1.36532396e-01,  2.99815297e-01,  6.81271181e-02, ...,
         0.00000000e+00,  3.00000000e+00,  1.35200000e+03],
       [-1.30906045e-01,  3.16850781e-01,  6.29489273e-02, ...,
         0.00000000e+00,  3.00000000e+00,  1.30000000e+01],
       [-1.36532396e-01,  2.99815297e-01,  6.81271181e-02, ...,
         0.00000000e+00,  3.00000000e+00,  1.06000000e+02],
       ...,
       [-1.20321102e-01,  3.11481416e-01,  5.85596636e-02, ...,
         0.00000000e+00,  3.00000000e+00,  5.00000000e+00],
       [-1.24281712e-01,  2.64844120e-01,  4.82976660e-02, ...,
         0.00000000e+00,  3.00000000e+00,  4.00000000e+00],
       [-1.26324266e-01,  2.77778417e-01,  4.48292866e-02, ...,
         0.00000000e+00,  3.00000000e+00,  7.00000000e+00]])

In [40]:
#Preparing data for modelling
X_train=feature_matrix
y=df["IS_HCP"]
# splitting data into train and validation
x_train,x_val,y_train, y_val =train_test_split(X_train, y, test_size=0.25, random_state=42)

x_test=feature_matrix1

In [41]:
# using RF classifier
rf=RandomForestClassifier(n_estimators=390,max_depth=20,class_weight='balanced',min_samples_split=11,min_samples_leaf=10,max_features='sqrt',max_leaf_nodes=50,n_jobs=-1,random_state=42)
rf.fit(x_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20, max_leaf_nodes=50,
                       min_samples_leaf=10, min_samples_split=11,
                       n_estimators=390, n_jobs=-1, random_state=42)

In [42]:
# predicting validation values
y_pred_val = rf.predict(x_val)
y_pred_val

array([1., 0., 0., ..., 0., 0., 0.])

In [43]:
# evaluating validation values

print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96     19928
         1.0       0.87      0.96      0.91      8556

    accuracy                           0.94     28484
   macro avg       0.92      0.95      0.93     28484
weighted avg       0.95      0.94      0.94     28484



In [44]:
# # predicting train and test values

train_pred=rf.predict(x_train)
y_pred=rf.predict(x_test)
train_pred

array([1., 0., 0., ..., 0., 1., 0.])

In [47]:
confusion_matrix(y_pred_val,y_val)

array([[18676,   375],
       [ 1252,  8181]], dtype=int64)

In [48]:
# CREATING FINAL TEST DATA FOR USER ID'S WITH PREDICTED VALUES
# index=test.index
test=test.reset_index()
final_df=pd.DataFrame(y_pred,index=test["ID"])
final_df.rename(columns={0:"is_hcp or not"},inplace=True)
final_df[final_df["is_hcp or not"]==1]
# Assign the predicted values to the DataFrame column
# predicted_df['Predicted_Value'] = predicted_test
# final_df
test.set_index('ID', inplace=True)
# final_df.set_index('ID', inplace=True)
final_df['KEYWORDS'] = test['PREPROCESSED_KEYWORDS']
final_df['KEYWORD_VECTORS'] = test['KEYWORD_VECTORS']

In [49]:
final_df[final_df["is_hcp or not"]==1]

,is_hcp or not,KEYWORDS,KEYWORD_VECTORS
ID,,,
115505,1.0,Critical Care|Emergency Medicine|General Pract...,"[-0.049428675, 0.33350885, 0.0895402, 0.217109..."
115509,1.0,Intensive Care|Shock|Ischemic Heart Disease|He...,"[-0.10041688, 0.38202363, 0.026872931, 0.32505..."
115511,1.0,Medicine|Transplantation|Clinical|Trauma|Total...,"[-0.11152679, 0.3705898, 0.09094602, 0.2220411..."
115513,1.0,Pharmacology|Resuscitation|General Practice|Ph...,"[-0.088133894, 0.35060117, 0.11854341, 0.25298..."
115516,1.0,Asthma|Female|Rheumatoid|Pathology|Neurology|P...,"[-0.18304862, 0.28207615, 0.037343763, 0.31948..."
...,...,...,...
143983,1.0,Pharmacology|Physicians|Technology|Oncology|Rh...,"[-0.12742351, 0.3199693, 0.09374208, 0.2610046..."
143984,1.0,Genetic|Large|Autoimmune|Health|False|Abortion...,"[-0.2356285, 0.2679312, 0.04846624, 0.0946987,..."
143990,1.0,Oncology|Medicine|Neurology|Ophthalmology|Pedi...,"[-0.18648109, 0.2932563, 0.06531716, 0.2778417..."


In [50]:
# FILLING TAXONOMIES WHERE KEYWORD MATCHES WITH THE TAXONOMY_MAP
final_df['taxonomy'] =final_df.apply(lambda row: taxonomy_map[row['KEYWORDS']] if row['is_hcp or not'] == 1 and row['KEYWORDS'] in taxonomy_map else 'no taxonomy', axis=1)
final_df

,is_hcp or not,KEYWORDS,KEYWORD_VECTORS,taxonomy
ID,,,,
115501,0.0,General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.122466646, 0.2960806, 0.12969866, 0.250681...",no taxonomy
115502,0.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,"[-0.07815745, 0.43258974, 0.020065375, 0.16374...",no taxonomy
115503,0.0,General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.122466646, 0.2960806, 0.12969866, 0.250681...",no taxonomy
115504,0.0,Dental|Total|Clinical|Pharmacology|Physicians|...,"[0.0250384, 0.4310842, 0.1060205, 0.27207673, ...",no taxonomy
115505,1.0,Critical Care|Emergency Medicine|General Pract...,"[-0.049428675, 0.33350885, 0.0895402, 0.217109...",no taxonomy
...,...,...,...,...
143989,0.0,General|Neurology|Ophthalmology|Medicine|Clini...,"[-0.15335903, 0.26359773, 0.07621166, 0.259216...",no taxonomy
143990,1.0,Oncology|Medicine|Neurology|Ophthalmology|Pedi...,"[-0.18648109, 0.2932563, 0.06531716, 0.2778417...",2084P0800X
143991,1.0,Diagnosis|Pathology|Neurological Surgery|Inten...,"[-0.07932103, 0.37711325, 0.15787539, 0.112127...",no taxonomy


In [51]:
final_df["taxonomy"].astype('str')

ID
115501    no taxonomy
115502    no taxonomy
115503    no taxonomy
115504    no taxonomy
115505    no taxonomy
             ...     
143989    no taxonomy
143990     2084P0800X
143991    no taxonomy
143992     207R00000X
143993    no taxonomy
Name: taxonomy, Length: 28493, dtype: object

In [52]:
keyword_vector1

{'General': array([-1.53359026e-01,  2.63597727e-01,  7.62116611e-02,  2.59216189e-01,
        -1.09721810e-01, -3.19298804e-01,  2.06216604e-01,  7.16670692e-01,
        -4.38138723e-01, -1.81095243e-01, -1.16925687e-02, -2.19912082e-01,
        -1.17437094e-01,  1.65260255e-01,  3.06070354e-02,  1.88617244e-01,
         4.53775406e-01, -8.14606696e-02, -1.13437094e-01, -5.65823793e-01,
        -5.78370951e-02,  3.73594463e-02, -1.09895609e-01, -1.42381161e-01,
        -3.14455539e-01,  2.08956987e-01, -2.41423815e-01, -2.65868623e-02,
        -8.21533501e-02,  1.05852187e-01,  3.27171177e-01, -1.06174439e-01,
         9.20763537e-02, -1.67192861e-01, -2.59081423e-01,  1.23459615e-01,
         1.31964490e-01,  7.13930577e-02, -4.10379767e-02, -2.86897123e-01,
         1.92068815e-01, -1.17957592e-01, -1.72745749e-01,  1.76445737e-01,
         6.88833743e-02, -1.86321080e-01, -3.77394736e-01, -3.17299535e-04,
         4.75306846e-02, -1.53287351e-01, -1.80419777e-02, -3.03123165e-02,
 

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(vector1, vector2):
    return cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))[0][0]

num_neighbors = 5  # Number of neighbors to consider

# Iterate over each row in the dataframe
for index, row in final_df.iterrows():
    if row['is_hcp or not'] == 1 and (pd.isnull(row['taxonomy']) or row['taxonomy'] == 'no taxonomy'):
        keyword_vector = row['KEYWORD_VECTORS']
        keyword_scores = {}
        
        # Calculate similarity scores with each keyword in the keyword_taxonomy_map
        for keyword, taxonomies in keyword_taxonomy_map.items():
            similarities = []
            if keyword in keyword_vector1:
                for taxonomy in taxonomies:
                    similarity = calculate_similarity(keyword_vector, keyword_vector1[keyword])
                    similarities.append(similarity)
            keyword_scores[keyword] = max(similarities) if similarities else 0.0
        
        # Sort keywords by similarity score in descending order
        sorted_keywords = sorted(keyword_scores, key=keyword_scores.get, reverse=True)
        
        # Find the top similar keywords with known taxonomies
        top_similar_keywords = [k for k in sorted_keywords if k in keyword_taxonomy_map][:num_neighbors]
        
        # Get the taxonomies of the top similar keywords
        top_taxonomies = [taxonomy for keyword in top_similar_keywords for taxonomy in keyword_taxonomy_map[keyword]]
        
        # Find the most common taxonomy among the top neighbors
        most_common_taxonomy = max(set(top_taxonomies), key=top_taxonomies.count)
        
        # Update the taxonomy for the current row
        final_df.at[index, 'taxonomy'] = most_common_taxonomy
        print(f"Updated Taxonomy for index {index}: {most_common_taxonomy}")
# INTERUPPTED AS IT TOOOK MORE TIME BUT IT WORKS.

Updated Taxonomy for index 115505: 207RC0000X
Updated Taxonomy for index 115511: 363L00000X
Updated Taxonomy for index 115513: 207R00000X
Updated Taxonomy for index 115516: 2084P0800X
Updated Taxonomy for index 115525: 207RA0001X
Updated Taxonomy for index 115527: 363L00000X
Updated Taxonomy for index 115530: 207RC0000X
Updated Taxonomy for index 115537: 363L00000X
Updated Taxonomy for index 115538: 207R00000X
Updated Taxonomy for index 115542: nan
Updated Taxonomy for index 115548: 207R00000X
Updated Taxonomy for index 115552: 207Q00000X
Updated Taxonomy for index 115557: 207R00000X
Updated Taxonomy for index 115560: 2084P0800X
Updated Taxonomy for index 115565: 2084P2900X
Updated Taxonomy for index 115568: 207R00000X
Updated Taxonomy for index 115578: 2080P0207X
Updated Taxonomy for index 115580: 2084P0800X
Updated Taxonomy for index 115598: 207R00000X
Updated Taxonomy for index 115601: nan
Updated Taxonomy for index 115605: 208000000X
Updated Taxonomy for index 115608: 2084P0800X
Up

KeyboardInterrupt: 

In [56]:
final_df[final_df["taxonomy"]=='no taxonomy'].head(50)

,is_hcp or not,KEYWORDS,KEYWORD_VECTORS,taxonomy
ID,,,,
115501,0.0,General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.122466646, 0.2960806, 0.12969866, 0.250681...",no taxonomy
115502,0.0,Bone Marrow|Radiography|Chronic|Oncology|Psych...,"[-0.07815745, 0.43258974, 0.020065375, 0.16374...",no taxonomy
115503,0.0,General|Endocrine|False|Medicine|Surgery|Urolo...,"[-0.122466646, 0.2960806, 0.12969866, 0.250681...",no taxonomy
115504,0.0,Dental|Total|Clinical|Pharmacology|Physicians|...,"[0.0250384, 0.4310842, 0.1060205, 0.27207673, ...",no taxonomy
115506,0.0,small|covid|nephrology|radiology|preventive me...,"[-0.00013506961, 0.09330168, 0.07144191, -0.41...",no taxonomy
115507,0.0,Transplantation|Technology|Chronic|Neoplasms|U...,"[-0.15836692, 0.26811698, 0.00616305, 0.168219...",no taxonomy
115508,0.0,Family Practice|Drainage|Clinical|Dermatology|...,"[-0.161582, 0.266964, 0.010480281, 0.20228106,...",no taxonomy
115510,0.0,chronic kidney disease|nephrology|radiology|da...,"[0.04010716, 0.053695057, -0.026530465, -0.402...",no taxonomy
115512,0.0,Family Practice|Drainage|Clinical|Dermatology|...,"[-0.161582, 0.266964, 0.010480281, 0.20228106,...",no taxonomy


In [58]:
final_df=final_df.drop('KEYWORD_VECTORS',axis=1)

In [59]:
final_df[final_df["taxonomy"]!="no taxonomy"].head(50)

,is_hcp or not,KEYWORDS,taxonomy
ID,,,
115505,1.0,Critical Care|Emergency Medicine|General Pract...,207RC0000X
115509,1.0,Intensive Care|Shock|Ischemic Heart Disease|He...,2084P0804X
115511,1.0,Medicine|Transplantation|Clinical|Trauma|Total...,363L00000X
115513,1.0,Pharmacology|Resuscitation|General Practice|Ph...,207R00000X
115516,1.0,Asthma|Female|Rheumatoid|Pathology|Neurology|P...,2084P0800X
115522,1.0,Gastrointestinal Tract|Female|Oncology|Medicin...,2084N0400X
115525,1.0,Small|Total|Urology|Seasonal|General|Thyroid C...,207RA0001X
115527,1.0,Premature|Chronic|Reproductive|Gastroenterolog...,363L00000X
115530,1.0,Rehabilitation|Transplantation|Clinical|Preven...,207RC0000X


In [57]:
final_df.drop(["KEYWORDS","taxonomy"],axis=1)

,is_hcp or not,KEYWORD_VECTORS
ID,,
115501,0.0,"[-0.122466646, 0.2960806, 0.12969866, 0.250681..."
115502,0.0,"[-0.07815745, 0.43258974, 0.020065375, 0.16374..."
115503,0.0,"[-0.122466646, 0.2960806, 0.12969866, 0.250681..."
115504,0.0,"[0.0250384, 0.4310842, 0.1060205, 0.27207673, ..."
115505,1.0,"[-0.049428675, 0.33350885, 0.0895402, 0.217109..."
...,...,...
143989,0.0,"[-0.15335903, 0.26359773, 0.07621166, 0.259216..."
143990,1.0,"[-0.18648109, 0.2932563, 0.06531716, 0.2778417..."
143991,1.0,"[-0.07932103, 0.37711325, 0.15787539, 0.112127..."


In [ ]:
final_df=final_df.to_csv("Final_output_file2.csv",index=True)

In [ ]:
# final_df[(final_df["taxonomy"]!="no taxonomy")]
# Bone Marrow|Radiography|Chronic|Oncology|Psychiatry|Intestine|Small Intestines|Physicians|Gastroenterology|Salivary Glands|Autoimmune|Hemolytic|False|Hereditary|Total|Cardiology|Surgery|Clinical|Dendritic Cells|Lung|Biopsy|General|Large|Small|Ophthalmology|Medicine|Lymphoid Tissue|Rheumatology|BLymphocytes|Stomach|Transplantation|Gastrointestinal Tract|Intestines|Diagnosis|Endocrine|Anemia|Refractory|Dermatology|Neurology|Urology
taxonomy_map

In [ ]:
# Performing k-fold cross-validation with multiple scoring metrics
cv_results1 = cross_val_score(rf, x_train, y_train, cv=5, scoring='accuracy')
cv_results2 = cross_val_score(rf, x_train, y_train, cv=5, scoring='precision')
# cv_results3= cross_val_score(rf, x_train, y_train, cv=5, scoring='recall')
# cv_results4 = cross_val_score(rf, x_train, y_train, cv=5, scoring='f1')

# mean_accuracy = cv_results1.mean()
# mean_precision = cv_results2.mean()
# mean_recall = cv_results3.mean()
# mean_f1_score = cv_results4.mean()

# std_accuracy = cv_results1.std()
# std_precision = cv_results2.std()
# std_recall = cv_results3.std()
# std_f1_score = cv_results4.std()

# print("Accuracy: {:.2f} (+/- {:.2f})".format(mean_accuracy, std_accuracy))
# print("Precision: {:.2f} (+/- {:.2f})".format(mean_precision, std_precision))
# print("Recall: {:.2f} (+/- {:.2f})".format(mean_recall, std_recall))
# print("F1 Score: {:.2f} (+/- {:.2f})".format(mean_f1_score, std_f1_score))
cv_results2

In [ ]:
def fill_taxonomy(hcp_instances, taxonomy_map):
    taxonomy_values = []

    for instance in hcp_instances:
        similar_keywords = get_similar_keywords(instance, taxonomy_map)

        # Calculate cumulative similarity scores for each taxonomy value
        taxonomy_scores = {}
        for keyword, taxonomy in similar_keywords.items():
            similarity_score = model.wv.similarity(instance, keyword)
            if taxonomy in taxonomy_scores:
                taxonomy_scores[taxonomy] += similarity_score
            else:
                taxonomy_scores[taxonomy] = similarity_score

        # Find the taxonomy value with the highest cumulative similarity score
        if taxonomy_scores:
            best_taxonomy = max(taxonomy_scores, key=taxonomy_scores.get)
            taxonomy_values.append((instance, best_taxonomy, taxonomy_scores[best_taxonomy]))

    return taxonomy_values

def get_similar_keywords(keywords, taxonomy_map):
    similar_keywords = {}
    for keyword in keywords.split('|'):
        if keyword in taxonomy_map:
            taxonomy = taxonomy_map[keyword]
            similar_keywords[keyword] = taxonomy

    return similar_keywords

# Assuming you have a list of HCP instances
hcp_instances = hcp_without_taxonomy[hcp_without_taxonomy['IS_HCP'] == 1]['PREPROCESSED_KEYWORDS'].tolist()

# Assuming you have a dataset with keywords and their taxonomy values
# Replace `keyword_taxonomy_map` with your actual keyword-to-taxonomy dictionary

# Fill taxonomy values for HCP instances
hcp_taxonomy_values = fill_taxonomy(hcp_instances, keyword_taxonomy_map)
print(hcp_taxonomy_values)
hcp_instances

In [ ]:
keyword_taxonomy_map

In [ ]:
# for i in taxonomy_map.values():
#     print(type(i))
# Check data type compatibility
if df['KEYWORDS'].dtype != str:
    df['KEYWORDS'] = df['KEYWORDS'].astype(str)

# Verify presence of exact matches
exact_matches = set(df['KEYWORDS']).intersection(taxonomy_map.keys())

print("Exact Matches Found:")
print(len(exact_matches))


In [ ]:
# Filter rows where HCP is identified but taxonomy is not populated
hcp_without_taxonomy = df[(df['TAXONOMY'].isnull())& (df['IS_HCP'] == True)]

# Filter rows where both HCP and taxonomy are identified
hcp_with_taxonomy = df[(df['IS_HCP'] == True) & (df['TAXONOMY'].notnull())]

# Filter rows where there is no information about HCP
unknown_hcp = df[df['IS_HCP'] != True]
hcp_with_taxonomy